In [0]:
!pip install tensorflow==1.14.0

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2

from PIL import Image

In [0]:
from google.colab import drive #STU NAPERVILLE ACCOUNTS DON'T WORK - USE YOUR PERSONAL GOOGLE ACCOUNTS
drive.mount('/content/drive')

In [0]:
#FORMATTING DATA SET (not tested)
#https://medium.com/@muskulpesent/create-numpy-array-of-images-fecb4e514c4b&sa=D&source=hangouts&ust=1582390259517000&usg=AFQjCNFIlPagqjM75QyaHiorDWqaazaL5g
train_images = []
train_labels = []
test_images = []
test_labels = []

#takes an exorbant amount of time to upload; may need to consider dataset pipelines or generators

files = glob.glob("/content/drive/My Drive/ROADREADER DATASETS/training/*.png")
for myFile in files:
  print(myFile)
  #image = cv2.imread(myFile)
  image = Image.open(myFile)
  image.load()
  image = np.asarray(image, dtype=np.float32)/255.0;
  train_images.append(image)
  train_labels.append(image[0])
train_labels = np.array(train_labels, dtype='float64')

files2 = glob.glob("/content/drive/My Drive/ROADREADER DATASETS/tests/*.png")
for myFile in files2:
  print(myFile)
  #image = cv2.imread(myFile)
  image = Image.open(myFile)
  image.load()
  image = np.asarray(image, dtype=np.float32)/255.0;
  test_images.append(image)
  test_labels.append(image[0])
test_labels = np.array(test_labels, dtype='float64')

#mnist = keras.datasets.mnist
#(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [13]:
print("shape before: " + str(np.shape(train_images)))
train_images = np.reshape(train_images, (-1, 300, 400, 3))
print("shape after: " + str(np.shape(train_images)))
test_images = np.reshape(test_images, (-1, 300, 400, 3))

shape before: (1187, 300, 400, 3)
shape after: (1187, 300, 400, 3)


In [5]:
'''
train_images = train_images / 255.
test_images = test_images / 255.
'''

'\ntrain_images = train_images / 255.\ntest_images = test_images / 255.\n'

In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(np.reshape(train_images[i], (300,400)), cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])
plt.show()

In [0]:
model = keras.Sequential([
  layers.Conv2D(14, 3, activation='relu', input_shape=(300,400,3)),
  layers.MaxPooling2D((2,2), padding='same'),
  layers.Conv2D(28, 3, activation='relu'),
  layers.MaxPooling2D((2,2), padding='same'),
  layers.Conv2D(42, 3, activation='relu'),
  layers.MaxPooling2D((2,2), padding='same'),
  layers.Flatten(),
  layers.Dense(20000, activation='relu'),
  layers.Dense(200, activation='relu'),
  layers.Dense(2, activation='softmax'), #classification layer; 0 for none 1 for stop
])
model.summary()

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.fit(x=train_images,y=train_labels, validation_data=(test_images, test_labels), epochs=10, batch_size=128)

model.save_weights('/content/model.h5')

In [0]:
predictions = np.argmax(model.predict(test_images), axis=1)
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(np.reshape(test_images[i], (28,28)), cmap=plt.cm.binary)
    plt.xlabel(predictions[i])
plt.show()